In [1]:
from IPython.display import display
from ipywidgets import FileUpload, Button, Dropdown, IntSlider, Output, VBox
from PIL import Image
import os

# === Widget UI ===
uploader = FileUpload(accept='.jpg,.jpeg,.png,.webp', multiple=True)
save_button = Button(description="💾 Save All")
convert_button = Button(description="🔁 Convert All")
compress_button = Button(description="🗜 Compress All")

format_dropdown = Dropdown(options=['jpg', 'png', 'webp'], description='Convert to:')
quality_slider = IntSlider(value=70, min=10, max=95, step=5, description='Quality:')

output = Output()

# === Folder setup ===
os.makedirs("input_images", exist_ok=True)
os.makedirs("output_images", exist_ok=True)

# === Variabel global ===
saved_image_paths = []

In [2]:
def on_save_all_clicked(b):
    global saved_image_paths
    output.clear_output()
    saved_image_paths = []

    if not uploader.value:
        with output:
            print("❌ Tidak ada file diupload.")
        return

    os.makedirs("input_images", exist_ok=True)

    for fileinfo in uploader.value:
        filename = fileinfo['name']
        ext = os.path.splitext(filename)[1].lower()

        if ext not in [".jpg", ".jpeg", ".png", ".webp"]:
            with output:
                print(f"❌ File {filename} tidak valid, dilewati.")
            continue

        filepath = os.path.join("input_images", filename)
        with open(filepath, "wb") as f:
            f.write(fileinfo['content'])

        saved_image_paths.append(filepath)

    with output:
        print(f"✅ {len(saved_image_paths)} gambar berhasil disimpan.")


In [3]:
from datetime import datetime

def create_output_subfolder(action: str) -> str:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    folder_name = f"{action}_{timestamp}"
    folder_path = os.path.join("output_images", folder_name)
    os.makedirs(folder_path, exist_ok=True)
    return folder_path

In [4]:
def format_bytes(size):
    # Format ukuran file ke kB atau MB
    for unit in ['B', 'KB', 'MB']:
        if size < 1024.0:
            return f"{size:.1f} {unit}"
        size /= 1024.0
    return f"{size:.1f} GB"

def print_file_comparison(before_path, after_path):
    before_size = os.path.getsize(before_path)
    after_size = os.path.getsize(after_path)
    saved = before_size - after_size
    percent = (saved / before_size) * 100 if before_size else 0

    print(f"📦 Sebelum : {format_bytes(before_size)}")
    print(f"📉 Sesudah : {format_bytes(after_size)}")
    print(f"💯 Efisiensi : {percent:.1f}% lebih kecil\n")

In [5]:
def on_convert_all_clicked(b):
    output.clear_output()

    if not saved_image_paths:
        with output:
            print("❌ Belum ada gambar yang disimpan.")
        return

    target_format = format_dropdown.value.lower()
    out_folder = create_output_subfolder(f"converted")

    with output:
        for path in saved_image_paths:
            img = Image.open(path)
            filename = os.path.splitext(os.path.basename(path))[0]
            new_path = os.path.join(out_folder, f"{filename}_converted.{target_format}")
            img.save(new_path, format=target_format.upper())
            print(f"✅ {filename} dikonversi ke {target_format.upper()}")
            print_file_comparison(path, new_path)

In [6]:
def on_compress_all_clicked(b):
    output.clear_output()

    if not saved_image_paths:
        with output:
            print("❌ Belum ada gambar yang disimpan.")
        return

    out_folder = create_output_subfolder(f"compressed")
    quality = quality_slider.value

    with output:
        for path in saved_image_paths:
            ext = os.path.splitext(path)[1].lower()
            if ext not in [".png",".jpg", ".jpeg", ".webp"]:
                print(f"⚠️ {os.path.basename(path)} dilewati (format tidak dikompres).")
                continue

            img = Image.open(path)
            filename = os.path.splitext(os.path.basename(path))[0]
            new_path = os.path.join(out_folder, f"{filename}_compressed{ext}")
            img.save(new_path, format=img.format, quality=quality, optimize=True)

            print(f"✅ {filename} dikompres (quality={quality}%)")
            print_file_comparison(path, new_path)

In [7]:
save_button.on_click(on_save_all_clicked)
convert_button.on_click(on_convert_all_clicked)
compress_button.on_click(on_compress_all_clicked)


In [8]:
display(VBox([
    uploader,
    save_button,
    format_dropdown,
    convert_button,
    quality_slider,
    compress_button,
    output
]))
